In [ ]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix


In [ ]:
# Dataset is assumed to be provided in the runtime environment as stated
books = pd.read_csv("books.csv")
ratings = pd.read_csv("ratings.csv")
users = pd.read_csv("users.csv")


In [ ]:
# Filter out users with < 200 ratings and books with < 100 ratings
user_counts = ratings['User-ID'].value_counts()
book_counts = ratings['ISBN'].value_counts()

ratings = ratings[ratings['User-ID'].isin(user_counts[user_counts >= 200].index)]
ratings = ratings[ratings['ISBN'].isin(book_counts[book_counts >= 100].index)]

# Merge ratings with books for title information
ratings_with_titles = ratings.merge(books, on='ISBN')
pivot = ratings_with_titles.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating').fillna(0)
matrix = csr_matrix(pivot.values)


In [ ]:
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(matrix)
book_titles = list(pivot.index)


In [ ]:
def get_recommends(book_title: str):
    if book_title not in pivot.index:
        return [book_title, []]

    book_index = pivot.index.get_loc(book_title)
    distances, indices = model.kneighbors([pivot.iloc[book_index].values], n_neighbors=6)
    recommendations = []

    for i in range(1, len(distances[0])):
        rec_title = book_titles[indices[0][i]]
        rec_dist = distances[0][i]
        recommendations.append([rec_title, rec_dist])

    return [book_title, recommendations]


In [ ]:
get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
